In [27]:
## imports here !!!
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

from sklearn.impute import SimpleImputer

In [28]:
'''
Importing data
'''
train_data= pd.read_csv('iith_foml_2020_train.csv')
train = pd.DataFrame(train_data)   ### Raw data
test_data= pd.read_csv('iith_foml_2020_test.csv')
test = pd.DataFrame(test_data)    ### Raw data
test.head()



#test.describe()

,Feature 1 (Discrete),Feature 2 (Discrete),Feature 3 (Discrete),Feature 4 (Discrete),Feature 5 (Discrete),Feature 6 (Discrete),Feature 7 (Discrete),Feature 8 (Discrete),Feature 9,Feature 10,Feature 11,Feature 12,Feature 13,Feature 14,Feature 15,Feature 16,Feature 17,Feature 18,Feature 19 (Discrete),Feature 20 (Discrete),Feature 21 (Discrete),Feature 22 (Discrete),Feature 23 (Discrete),Feature 24
0,146,12,42,14,7,1,1,1,118.004,35693.500,37872.100,0.025,96.405,1435.935,4200.300,1900.100,nan,nan,44,12,2,42,146,37384.500
1,35,0,12,5,0,0,1,0,0.001,471.500,486.400,0.001,97.285,94.181,531.400,nan,nan,nan,17,6,8,0,35,41465.100
2,1018,8,259,2,1,1,1,1,nan,35774.500,35798.500,0.000,0.023,1436.092,5514.200,nan,nan,15.040,1,1,3,20,1018,37826.200
3,383,7,117,5,1,1,1,1,53.002,34094.100,37636.200,0.042,0.531,1440.095,3358.400,1100.200,14000.001,15.040,101,6,3,20,383,40277.300
4,1216,7,40,5,2,0,1,4,0.005,1471.300,1495.400,0.002,82.502,115.504,225.100,nan,nan,3.020,276,6,7,43,1221,28419.500


### Analysis of missing values...


In [29]:

def Summarise_missing_values(new_data):
    cols_with_missing = (col for col in new_data.columns if new_data[col].isnull().any())
    num_missing = (new_data[cols_with_missing].isnull()).sum()
    print("Missing values in percent")
    # report the results
    print(round((num_missing/new_data.shape[0]) * 100, 2))
    print("######################################################################################") 

print("****************************For TRAIN data")
Summarise_missing_values(train.copy())
print("****************************For Test data")
Summarise_missing_values(test.copy())


****************************For TRAIN data
Missing values in percent
Feature 9     1.410
Feature 10    0.100
Feature 11    0.100
Feature 12    0.100
Feature 13    0.100
Feature 14    0.100
Feature 15    7.240
Feature 16   67.300
Feature 17   54.930
Feature 18   33.200
Feature 24    0.100
dtype: float64
######################################################################################
****************************For Test data
Missing values in percent
Feature 9     0.940
Feature 14    0.470
Feature 15    7.280
Feature 16   65.490
Feature 17   52.820
Feature 18   26.760
dtype: float64
######################################################################################


###  Filling missing values by imputation through mean...

In [30]:

def imputing_missing_values(df):
    data = df.to_numpy()
    # define the imputer
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    imputer  = imputer.fit(data)
    # transform the dataset
    transformed_values = imputer.transform(data)
    imputed_data = pd.DataFrame(data = transformed_values,columns = df.columns) 
    return imputed_data


L1_train = imputing_missing_values(train)
L1_test = imputing_missing_values(test)

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', None)
L1_train.describe()
#imputed_test.describe()

,Feature 1 (Discrete),Feature 2 (Discrete),Feature 3 (Discrete),Feature 4 (Discrete),Feature 5 (Discrete),Feature 6 (Discrete),Feature 7 (Discrete),Feature 8 (Discrete),Feature 9,Feature 10,Feature 11,Feature 12,Feature 13,Feature 14,Feature 15,Feature 16,Feature 17,Feature 18,Feature 19 (Discrete),Feature 20 (Discrete),Feature 21 (Discrete),Feature 22 (Discrete),Feature 23 (Discrete),Feature 24,Target Variable (Discrete)
count,994.000,994.000,994.000,994.000,994.000,994.000,994.000,994.000,994.000,994.000,994.000,994.000,994.000,994.000,994.000,994.000,994.000,994.000,994.000,994.000,994.000,994.000,994.000,994.000,994.000
mean,708.187,5.899,159.564,11.651,2.624,0.581,2.041,1.643,6.113,14527.974,16057.137,0.020,50.015,642.128,2042.520,1329.780,3814.421,2061.807,94.004,7.660,4.777,45.635,709.334,34605.373,1.706
std,405.826,7.563,106.706,15.159,2.652,0.747,2.550,1.532,54.273,16438.642,18967.035,0.111,41.766,659.089,2201.290,722.723,3117.065,7582.518,90.311,10.658,4.774,36.070,407.361,6117.763,2.417
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-177.116,200.400,322.500,0.000,0.001,7.014,0.200,6.500,4.055,0.270,0.000,0.000,0.000,0.000,0.000,7530.300,0.000
25%,357.750,0.000,66.250,2.000,1.000,0.000,1.000,0.000,0.001,621.300,683.775,0.000,0.094,97.702,207.150,1329.780,1615.004,8.030,12.000,1.000,1.000,14.250,357.750,28378.375,0.250
50%,705.000,3.000,146.000,5.000,2.000,0.000,1.000,1.000,0.003,1409.200,1415.150,0.001,54.904,114.101,1480.450,1329.780,3814.421,15.020,73.500,2.000,3.000,39.000,705.000,37264.750,1.000
75%,1064.500,8.000,239.000,15.000,3.000,1.000,1.000,3.000,0.005,35770.400,35794.500,0.002,97.470,1436.083,3200.375,1329.780,3814.421,2061.807,152.000,12.000,7.000,64.000,1064.500,40038.150,2.000
max,1412.000,46.000,386.000,80.000,14.000,4.000,19.000,7.000,328.502,37778.400,156833.300,0.864,143.402,4032.863,18000.300,10000.400,20000.005,44118.010,318.000,58.000,23.000,155.000,1417.000,41634.300,17.000


### Normalising data : Min-max 

In [31]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result


### Remove outliers..

In [109]:
#### Removing outliers..


def remove_outliers(df,r = 5.5):
    outfree = pd.DataFrame()
    outfree = df[(np.abs(stats.zscore(df)) < r).all(axis=1)]
    outfree.index = range(len(outfree))
    return outfree

In [114]:
'''
Dividing into X, Y
'''


L2_train = L1_train.drop(columns=['Feature 16', 'Feature 17','Feature 13','Feature 11','Feature 9'],axis = 1)
L2_test = L1_test.drop(columns=['Feature 16', 'Feature 17','Feature 13','Feature 11','Feature 9'],axis = 1)

L3_train = remove_outliers(L2_train)
L3_test = L2_test


#################################################################################

X = L3_train.drop(columns=['Target Variable (Discrete)'],axis = 1)
Y = L3_train[['Target Variable (Discrete)']]
X_test = L3_test

print(train.shape)
print(X.shape)
Y.describe()

#Summarise_missing_values(Y.copy())
### Normalising...
#X = normalize(X)
#X_test = normalize(X_test)

(994, 25)
(932, 19)


,Target Variable (Discrete)
count,932.000
mean,1.637
std,2.194
min,0.000
25%,1.000
50%,1.000
75%,2.000
max,14.000


In [111]:
L3_train.tail()


,Feature 1 (Discrete),Feature 2 (Discrete),Feature 3 (Discrete),Feature 4 (Discrete),Feature 5 (Discrete),Feature 6 (Discrete),Feature 7 (Discrete),Feature 8 (Discrete),Feature 10,Feature 12,Feature 14,Feature 15,Feature 18,Feature 19 (Discrete),Feature 20 (Discrete),Feature 21 (Discrete),Feature 22 (Discrete),Feature 23 (Discrete),Feature 24,Target Variable (Discrete)
927,1388.000,0.000,382.000,14.000,2.000,0.000,1.000,0.000,628.500,0.002,97.574,2700.400,2061.807,104.000,12.000,11.000,53.000,1393.000,41473.400,0.000
928,1339.000,0.000,344.000,14.000,3.000,0.000,1.000,0.000,520.300,0.001,95.235,1.300,2061.807,205.000,12.000,10.000,63.000,1344.000,40916.200,2.000
929,527.000,7.000,40.000,5.000,5.000,2.000,1.000,3.000,19130.100,0.001,676.384,1415.500,7.020,101.000,6.000,7.000,95.000,527.000,37938.400,6.000
930,1144.000,12.000,120.000,14.000,3.000,0.000,1.000,0.000,785.200,0.001,100.805,300.100,2061.807,166.000,12.000,11.000,53.000,1149.000,33433.500,0.000
931,1284.000,0.000,346.000,14.000,0.000,0.000,1.000,0.000,466.300,0.001,94.105,3.300,2061.807,289.000,12.000,11.000,134.000,1289.000,40927.100,2.000


In [117]:

'''
Training with stratified K-fold
'''

i=1
kf = StratifiedKFold(n_splits=5,random_state=1,shuffle=True)

param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }



for train_index,test_index in kf.split(X,Y):
    print('\n{} of kfold {}'.format(i,kf.n_splits))
    xtr,xvl = X.loc[train_index],X.loc[test_index]
    ytr,yvl = Y.loc[train_index],Y.loc[test_index]

    model =GridSearchCV(XGBClassifier(objective='multi:softmax'), param_grid, cv=5, scoring= 'f1_macro',iid=True, )
    model.fit(xtr, ytr)

    print (model.best_params_)
    y_pred=model.predict(xvl)
    y_pred = [round(value) for value in y_pred]
    
    
    print('accuracy_score',accuracy_score(yvl,y_pred) * 100)
    i+=1


/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(


1 of kfold 5


/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'colsample_bytree': 0.8, 'gamma': 0.5, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.8}
accuracy_score 90.9090909090909

2 of kfold 5


/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


...

{'colsample_bytree': 0.8, 'gamma': 1, 'max_depth': 4, 'min_child_weight': 1, 'subsample': 0.8}
accuracy_score 90.86021505376344

4 of kfold 5


/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'colsample_bytree': 1.0, 'gamma': 0.5, 'max_depth': 3, 'min_child_weight': 1, 'subsample': 0.8}
accuracy_score 90.86021505376344

5 of kfold 5


/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'colsample_bytree': 1.0, 'gamma': 0.5, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.6}
accuracy_score 94.08602150537635


In [118]:
def predict_and_submission(test_data,model,title):### test_data in pd format
    test = pd.DataFrame(test_data)
    predictions = model.predict(test)

    output_test_data = pd.DataFrame() 
    output_test_data['Category'] = predictions.astype(int)

    output_test_data['Id'] = list(np.arange(1,predictions.size+1))

    submission = output_test_data[['Id','Category']]
    submission.to_csv(title, index=False)
    submission.head()
    return submission

##### Plsss change the title........
output = predict_and_submission(X_test,model,title = "S_007.csv")
output.head()

,Id,Category
0,1,6
1,2,0
2,3,1
3,4,1
4,5,1


In [101]:
output.shape

(426, 2)